In [1]:
import socketio

# Create a SocketIO client
sio = socketio.Client()

# Connect to the server
@sio.event
def connect():
    print('Connected to server')

# Define a function to send a message to the server
def send_message_to_server(message):
    sio.emit('message_from_notebook', message)
# Define a function to send the CSV data to the server
def send_csv_data(csv_data):
    sio.emit('csv_data', csv_data)
# Connect to the server
sio.connect('http://localhost:5000')

# Send a message to the server
send_message_to_server('Notebook Connected!')

#csv_data = '"Name","Age","City","Country","Job","Salary","Email","Phone","Birthday","Height","Weight"'
#send_csv_data(csv_data)

#send_csv_data(csv_data)

Connected to server


In [2]:
import sys
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from flask import flash
from time import sleep
# Load environment variables from a .env file
load_dotenv()

# Retrieve the OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4o"  # or "llama3"

# Choose the model and embeddings based on the MODEL variable
if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

# Initialize the output parser
parser = StrOutputParser()

# Define the prompt template
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}  
"""

prompt = PromptTemplate.from_template(template)

# Load and process the MMD file content
def load_mmd_content(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Test the function with a sample question
question = "find and extract all the tabular content inside the file to csv format, if you find more than one table give me all and just respond with the csv format straight away with their headers?"

# Get paths from environment variables
UPLOAD_FOLDER = os.environ.get('UPLOAD_FOLDER', 'uploads')
PROCESSED_FOLDER = os.environ.get('PROCESSED_FOLDER', 'processed')
OUTPUT_FOLDER = os.environ.get('OUTPUT_FOLDER', 'uploads/output')
NOUGAT_PDF_FILENAME = os.environ.get('NOUGAT_PDF_FILENAME', '')

# Get the filename from the environment variable
filename = os.environ.get('NOUGAT_PDF_FILENAME', '')
#filename = 'park.pdf'

# Construct the path to the MMD file
# Construct the path to the MMD file
mmd_file_path = os.path.join(OUTPUT_FOLDER, filename.replace('.pdf', '.mmd'))
#mmd_file_path = 'uploads/output/park.mmd'

# Process the content
print("Loading MMD content")
#emit('progress', {'message': 'Loading MMD content'})

# Load the MMD content
mmd_content = load_mmd_content(mmd_file_path)

# Define the chain
chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=parser
)

# Create a function to get answers with the MMD content as context
def get_answer_with_mmd_context(question, mmd_content):
    # Format the prompt with the MMD content as context
    formatted_prompt = prompt.format(context=mmd_content, question=question)
    
    # Invoke the chain with the formatted prompt
    answer = chain.invoke({"context": mmd_content, "question": question})
    
    return answer

# Test the function with a sample question
answer = get_answer_with_mmd_context(question, mmd_content)
#print(answer['text'])  # Print the answer text in terminal

# Check if the answer is "I don't know"
if answer['text'] == "I don't know":
    print("Notebook finished: No tables found")  # Indicate failure to Flask app
    send_message_to_server(f"Processed PDF, No tables found in {NOUGAT_PDF_FILENAME}")



send_message_to_server(f"Found and extracted tables from {NOUGAT_PDF_FILENAME}")

# Process the content
print("Processing content")
#emit_progress("Notebook execution started.")
#emit('progress', {'message': ' lolzz '})


# Print environment variables
#send_message_to_server(f"UPLOAD_FOLDER: {UPLOAD_FOLDER}")
#send_message_to_server(f"PROCESSED_FOLDER: {PROCESSED_FOLDER}")
#send_message_to_server(f"OUTPUT_FOLDER: {OUTPUT_FOLDER}")
#send_message_to_server(f"NOUGAT_PDF_FILENAME: {NOUGAT_PDF_FILENAME}")
# Send a message to the server
#send_message_to_server('Notebook Connected!')


Loading MMD content


c:\windows\system32\project\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Processing content


In [3]:
# Construct the CSV file path dynamically using the filename variable
csv_file_path = os.path.join('output', f'{filename.replace(".pdf", ".csv")}')
#csv_file_path = 'output/park.csv'
# Check if the answer is "I don't know"
if answer['text'] == "I don't know":
    print("No tables found")
    send_message_to_server('No tables found')
    #raise ValueError("No tables found")
# Remove leading and trailing triple backticks and 'csv'
csv_data = answer['text'].strip('```').replace('csv', '').replace('```', '')
#csv_data = answer['text']
send_csv_data(csv_data)

# Continue with the subsequent code cells
# Write the CSV data to the dynamically constructed file path
with open(csv_file_path, 'w', encoding='utf-8') as file:
    file.write(csv_data)
#print(answer['text'])
# Instead of print(answer['text']), use:
print(f"CSV data has been written to {csv_file_path} file.")
print(csv_data)
send_message_to_server(f"CSV data has been written to {csv_file_path} file")

CSV data has been written to output\park.csv file.

Method,App.,Material,Solid,µ,V,h0,fr,Temp.,w,tvaridity
D(21),DSSC,TiO2,10wt%,-,1,-,-,120-140,-,10 (wet)
DG(22),**-**,**-**,992-1088 kg/m3,1-4,19.8-210,-,-,-,100,1-10 (dry)
K(28),e-paper,TiO2,-,-,9.14,-,-,179-192,**-**,12-40 (dry)
K(37),OPV,ZnO,50 mg/ml,-,0.8,300,-,125-130,250,0.02 (dry)
K/MG(33),Battery (Li),Carbon,-,-,0.25-10,25 (K),-,875-1185,2-200 (dry)
K/SD(34),Fuel cell,PBI,9wt%,-,0.20,400,-,140,250,40 (dry)
LFS(24),Paper,TiO2,50 mg/ml,-,150,10^3-25-10^4,23,75-110,50,1 (dry)
MG(32),Sensor (flex),Carbon,1120 kg/m3,20,0.3 (web),-,-,130,150,4.31 (dry)
MG(31),TCF,PEDOT:PSS,-,1-1000,0.18,-,-,-,-,0.02 (dry)
SD(6),AR,Si,1.5-5,3.3,1.6-4.4,100,2-13,140,100,0.06-0.13 (dry)
SD(73),OPV,P3HT:PCBM,13.5 mg/ml,-,2,-,1.7,140,<305,0.129 (dry)
SD(88),OPV,,60 mg/ml,25,240,-,-,140,**-**,100 (wet)
SD(33),,Ag,-,-,1.2-2.4,100,2-7,**-**,250,0.67-0.252 (dry)
SD(35),PSA,PSA,20wt%,1800,2-10,100-400,10.65-53.74,120,150,4.5-5 (dry)
SD(34),TCF,,0.5wt%,**-**,2.

In [4]:
import os
import pandas as pd
import re
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Function to clean CSV file
def clean_cell(cell):
    if isinstance(cell, str):
        # Remove superscripts and subscripts like in DG\({}^{22}\) and TiO\({}_{2}\)
        cell = re.sub(r'\\?\({}(\^)\{[^}]*\}\)', '', cell)
        # Remove any remaining parentheses and their contents
        cell = re.sub(r'\([^)]*\)', '', cell)
        # Remove any leftover backslashes
        cell = cell.replace('\\', '')
        return cell.strip()
    return cell

def clean_csv(input_file):
    try:
        # Try to read the CSV file with utf-8 encoding
        df = pd.read_csv(input_file, encoding='utf-8')
    except UnicodeDecodeError:
        # If there's a UnicodeDecodeError, try with a different encoding
        try:
            df = pd.read_csv(input_file, encoding='latin1')
        except Exception as e:
            print(f"Error reading {input_file}: {e}")
            return
    except pd.errors.EmptyDataError:
        print(f"Warning: {input_file} is empty or unreadable.")
        return
    except Exception as e:
        print(f"Error processing {input_file}: {e}")
        return

    # Drop empty rows
    df.dropna(how='all', inplace=True)
    
    # Check if 'Method' column exists
    if 'Method' in df.columns:
        # Apply the cleaning function only to the 'Method' column
        df['Method'] = df['Method'].apply(clean_cell)

    try:
        # Write the cleaned data to the new CSV file
        df.to_csv(input_file, index=False, encoding='utf-8')
        print(f"Cleaned data saved to {input_file}")
    except Exception as e:
        print(f"Error saving {input_file}: {e}")

# Path to the folder containing CSV files
csv_folder_path = 'output'
output_folder_path = 'output'

# Get the NOUGAT_PDF_FILENAME from the environment variable
nougat_pdf_filename = os.environ.get('NOUGAT_PDF_FILENAME', '')
#nougat_pdf_filename = 'park.pdf'

# Construct the new CSV file path by stripping the .pdf extension and adding .csv
new_csv_path = os.path.join(csv_folder_path, os.path.splitext(nougat_pdf_filename)[0] + '.csv')
clean_csv(new_csv_path)

print("CSV cleaning complete.")
#send_message_to_server(f"Cleaned the uploaded CSV file: {new_csv_path}")

Cleaned data saved to output\park.csv
CSV cleaning complete.


In [5]:
import os
import shutil
import pandas as pd
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOllama, ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Load environment variables from a .env file
load_dotenv()

# Retrieve the OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4o" #"llama2"  # or "gpt-4"

# Initialize the model and embeddings
if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = ChatOllama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

# Define the prompt template
template = """
Answer the question based on the context below. 

Context: {context}

Question: {question}  
"""

prompt = PromptTemplate(
    input_variables=["context", "question"], 
    template=template
)

# Function to load CSV content
def load_csv_content(file_path):
    encodings = ['utf-8', 'latin1', 'iso-8859-1']
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            return df.to_csv(index=False)
        except UnicodeDecodeError:
            print(f"Error decoding {file_path} with {encoding}, trying next encoding.")
    raise ValueError(f"Unable to decode file {file_path} with available encodings.")

# Path to the folder containing CSV files
csv_folder_path = 'output'
output_folder_path = 'output'
specifications_folder_path = os.path.join(output_folder_path, 'specifications')

# Create the specifications folder if it doesn't exist
os.makedirs(specifications_folder_path, exist_ok=True)

# Pipeline the prompt with the model and parser
pipeline = prompt | model | StrOutputParser()

# Create a function to get answers with the CSV content as context
def get_answer_with_csv_context(question, csv_content):
    # Invoke the pipeline with the context and question
    answer = pipeline.invoke({"context": csv_content, "question": question})
    print(answer)
    return answer

# Define the question to check for specifications
question = "Does the CSV data contain specifications, just reply with yes or no?"

# Process each CSV file in the folder
for file_name in os.listdir(csv_folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(csv_folder_path, file_name)
        try:
            csv_content = load_csv_content(file_path)
            
            # Get the answer from the LLM
            answer = get_answer_with_csv_context(question, csv_content)
            print(answer)  # Remove the parentheses here
            # If the answer is "Yes", move the file to the specifications folder
            if 'yes' in answer.lower():
                shutil.move(file_path, os.path.join(specifications_folder_path, file_name))
            else:
                print("No specifications found in", file_name)
                send_message_to_server(f"No specifications found in {file_name}")

        except ValueError as e:
            print(e)
#send_csv_data(csv_data)

print("Processing complete.")
send_message_to_server(f"Idenified and moved files with specifications to {specifications_folder_path} folder.")


c:\windows\system32\project\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\windows\system32\project\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Yes
Yes
Processing complete.


In [6]:
import os
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
import pandas as pd

# Load environment variables from a .env file
load_dotenv()

# Retrieve the OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"  # or another OpenAI model
OUTPUT_FOLDER = 'output'
OUTPUT_SPECS_FOLDER = 'output/specifications'

# Initialize the model and embeddings
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
embeddings = OpenAIEmbeddings()

# Define the prompt template for understanding and combining CSV data
template = """\
You are an expert in data analysis. I have provided you with the contents of the combined CSV file and a new CSV file. Your task is to:

1. Understand the data in the combined CSV file and the new CSV file thoroughly.
2. Identify similar or related data between these two files.
3. Combine and consolidate the data from the new CSV file into the combined dataset.
4. Provide the updated combined data in CSV format, with appropriate headers.

Here are the contents of the combined CSV file:

{combined_csv_contents}

And here are the contents of the new CSV file:

{new_csv_contents}

Now, analyze these files, combine the new data into the combined dataset, if ou find data not compatible with each other then have different headers for both and provide the updated result in CSV format straight away starting from '''
"""

prompt = PromptTemplate(input_variables=["combined_csv_contents", "new_csv_contents"], template=template)

def handle_bad_line(line):
    print(f"Skipping bad line: {line}")

def load_csv_content(file_path):
    encodings = ['utf-8', 'latin1']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                content = f.read().strip()
            if not content:
                print(f"The file {file_path} is empty.")
                return ""
            df = pd.read_csv(file_path, encoding=encoding, engine='python', on_bad_lines='skip')
            return df.to_csv(index=False)
        except (UnicodeDecodeError, pd.errors.ParserError):
            print(f"Error decoding {file_path} with {encoding}, trying next encoding.")
    raise ValueError(f"Unable to decode file {file_path} with available encodings.")

# Get the NOUGAT_PDF_FILENAME from the environment variable
nougat_pdf_filename = os.environ.get('NOUGAT_PDF_FILENAME', '')
#nougat_pdf_filename = 'park.pdf'

# Construct the new CSV file path by stripping the .pdf extension and adding .csv
new_csv_path = os.path.join(OUTPUT_FOLDER, os.path.splitext(nougat_pdf_filename)[0] + '.csv')
#new_csv_path = 'output/specifications/park.csv'
if os.path.isfile(new_csv_path):
    new_csv_contents = load_csv_content(new_csv_path)
else:
    new_csv_path = os.path.join(OUTPUT_SPECS_FOLDER, os.path.splitext(nougat_pdf_filename)[0] + '.csv')
    if os.path.isfile(new_csv_path):
        new_csv_contents = load_csv_content(new_csv_path)
    else:
        raise FileNotFoundError(f"Could not find the CSV file for '{nougat_pdf_filename}' in either '{OUTPUT_FOLDER}' or '{OUTPUT_SPECS_FOLDER}'")

# Load the combined CSV file or create a new one
combined_csv_path = os.path.join(OUTPUT_SPECS_FOLDER, "mega_combined.csv")
if os.path.isfile(combined_csv_path):
    combined_csv_contents = load_csv_content(combined_csv_path)
    if not combined_csv_contents:
        print("The combined CSV file is empty. Using the new CSV contents.")
        combined_csv_contents = new_csv_contents
else:
    combined_csv_contents = new_csv_contents

# If the combined CSV file doesn't exist or is empty, create it with the new CSV contents
if not os.path.isfile(combined_csv_path) or not combined_csv_contents:
    with open(combined_csv_path, 'w', encoding='utf-8') as f:
        f.write(new_csv_contents)
    print(f"Created new combined CSV file at {combined_csv_path} with the contents of {new_csv_path}")
    send_message_to_server(f"Created new combined CSV file at {combined_csv_path} with the contents of {new_csv_path}")
    # Exit the script as we've just created the combined file with new contents
    exit()

# Use GPT to understand and combine the CSV data if the combined CSV file exists
if os.path.isfile(combined_csv_path):
    pipeline = prompt | model | StrOutputParser()
    combined_csv = pipeline.invoke({"combined_csv_contents": combined_csv_contents, "new_csv_contents": new_csv_contents})
    # Strip leading and trailing triple quotes if present
    if combined_csv.startswith("''''"):
        combined_csv = combined_csv[3:-3].strip()
        print("stripped triple quotes")
        
    # Save the updated combined CSV
    updated_csv_path = os.path.join(OUTPUT_SPECS_FOLDER, "mega_combined.csv")
    with open(updated_csv_path, 'w', encoding='utf-8') as f:
        f.write(combined_csv)

    print(f"Updated combined CSV data saved to {updated_csv_path}")
    send_message_to_server(f"Updated combined CSV data saved to {updated_csv_path}")

Updated combined CSV data saved to output/specifications\mega_combined.csv


In [7]:
import json
import requests
import time
# Trigger server restart
# Trigger server restart
time.sleep(50)
try:
    requests.post('http://localhost:5000/restart')
    print("Restart signal sent to server")
except requests.exceptions.ConnectionError:
    print("Server is restarting...")

# Wait for server to come back online
server_up = False
while not server_up:
    try:
        response = requests.get('http://localhost:5000/healthcheck')
        if response.status_code == 200:
            server_up = True
            print("Server is back online")
    except requests.exceptions.ConnectionError:
        print("Waiting for server to restart...")
        time.sleep(5)

Server is restarting...


Waiting for server to restart...


Server is back online
